In [1]:
import pandas as pd
import numpy as np
import re
import math
import copy

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
html = 'schedule/s2014/private.php'
fetched_dataframes = pd.io.html.read_html(html)
p_name = html.split('/')[1] + '_' + html.split('/')[2].split('.')[0]

In [10]:
lst = []
for i in range(len(fetched_dataframes)):
    if fetched_dataframes[i][0][0] == '種目':
        lst.append(i)

In [11]:
j = lst[0]
fetched_dataframes[j]

,0,1,2,3
0,種目,選手,記録,備考
1,1500m,松井 将器,"3'57""28",PB/歴代4位
2,柴田 幸樹,DNS,-,NaN
3,5000m,鬼頭 清仁,"18'20""04",-


In [12]:
k = lst[-1]
fetched_dataframes[k]

,0,1,2,3
0,種目,選手,記録,備考
1,10000ｍ,宮崎 晃年,"32'12""49",PB/歴代10位
2,鬼頭 清仁,DNS,-,NaN


In [13]:
dfs = pd.concat([fetched_dataframes[i].iloc[1:, :] for i in lst])
dfs

,0,1,2,3,4
1,1500m,松井 将器,"3'57""28",PB/歴代4位,NaN
2,柴田 幸樹,DNS,-,NaN,NaN
3,5000m,鬼頭 清仁,"18'20""04",-,NaN
1,5000m,大竹 康仁,"18'20""79",-,NaN
2,辻 航平,"16'32""20",-,NaN,NaN
3,小林 雅彦,DNS,-,NaN,NaN
4,松井 将器,"14'21""57",PB/工大新記録,NaN,NaN
1,100m予選,荒川 雄斗,"11""03",+1.7,-
2,米川 晴義,"11""10",+2.3,-,NaN
3,関 駿太,"10""99",+2.8,-,NaN


In [14]:
dfs.columns=['種目', '氏名', '記録', '風', '備考']
dfs = dfs.reset_index(drop=True)

dfs

,種目,氏名,記録,風,備考
0,1500m,松井 将器,"3'57""28",PB/歴代4位,NaN
1,柴田 幸樹,DNS,-,NaN,NaN
2,5000m,鬼頭 清仁,"18'20""04",-,NaN
3,5000m,大竹 康仁,"18'20""79",-,NaN
4,辻 航平,"16'32""20",-,NaN,NaN
5,小林 雅彦,DNS,-,NaN,NaN
6,松井 将器,"14'21""57",PB/工大新記録,NaN,NaN
7,100m予選,荒川 雄斗,"11""03",+1.7,-
8,米川 晴義,"11""10",+2.3,-,NaN
9,関 駿太,"10""99",+2.8,-,NaN


In [15]:
new_dfs = pd.DataFrame(columns=['種目', '氏名', '記録', '風', '備考'])

for i in range(len(dfs)):
    line = dfs.iloc[i]['種目']
    temp_df = pd.DataFrame(dfs.iloc[i])
    if re.search('\d', line) or re.search('跳', line) or re.search('投', line) or re.search('マラソン', line):
        temp_df = temp_df.T
    else:
        temp_df = temp_df.shift(1).T
    
    new_dfs = new_dfs.append(temp_df)
    
new_dfs

,種目,氏名,記録,風,備考
0,1500m,松井 将器,"3'57""28",PB/歴代4位,NaN
1,NaN,柴田 幸樹,DNS,-,NaN
2,5000m,鬼頭 清仁,"18'20""04",-,NaN
3,5000m,大竹 康仁,"18'20""79",-,NaN
4,NaN,辻 航平,"16'32""20",-,NaN
5,NaN,小林 雅彦,DNS,-,NaN
6,NaN,松井 将器,"14'21""57",PB/工大新記録,NaN
7,100m予選,荒川 雄斗,"11""03",+1.7,-
8,NaN,米川 晴義,"11""10",+2.3,-
9,NaN,関 駿太,"10""99",+2.8,-


In [16]:
new_dfs2 = copy.deepcopy(new_dfs)
track = pd.DataFrame(columns=['種目', '氏名', '記録', '風', '備考'])
field = pd.DataFrame(columns=['種目', '氏名', '記録', '風', '備考'])

status = False
for i in range(len(new_dfs2)):
    discipline = new_dfs2.iloc[i]['種目']
    if isinstance(discipline, str):
        if re.search('投', discipline) or re.search('跳', discipline):
            field = field.append(pd.DataFrame(new_dfs2.iloc[i]).T)
            status = True
            
        else:
            track = track.append(pd.DataFrame(new_dfs2.iloc[i]).T)
            status = False
    
    else:
        if math.isnan(discipline) and status:
            field = field.append(pd.DataFrame(new_dfs2.iloc[i]).T)
            status = True
        
        else:
            track = track.append(pd.DataFrame(new_dfs2.iloc[i]).T)
            status = False

In [17]:
track['時間'] = [np.nan for i in range(len(track))]
track['分'] = [np.nan for i in range(len(track))]
track['秒']  = [np.nan for i in range(len(track))]
track['秒以下'] = [np.nan for i in range(len(track))]

In [18]:
track = track.reset_index(drop=True)
field = field.reset_index(drop=True)

In [20]:
track2 = copy.deepcopy(track)

for i in range(len(track2)):
    line = track2.iloc[i]['記録']

    if re.search(':', line):
        track2['時間'][i] = line.split(':')[0]
        track2['分'][i] = line.split(':')[1].split('\'')[0]
        track2['秒'][i] = line.split(':')[1].split('\'')[1].split('\"')[0]
        
    elif re.search('\'', line):
        track2['分'][i] = line.split('\'')[0]
        track2['秒'][i] = line.split('\'')[1].split('\"')[0]
        if re.search('\"', line):
            track2['秒以下'][i] = line.split('\'')[1].split('\"')[1]
    
    elif re.search('\"', line):
        track2['秒'][i] = line.split('\"')[0]
        track2['秒以下'][i] = line.split('\"')[1]
        
track2

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/si

,種目,氏名,記録,風,備考,時間,分,秒,秒以下
0,1500m,松井 将器,"3'57""28",PB/歴代4位,NaN,NaN,3.0,57,28.0
1,NaN,柴田 幸樹,DNS,-,NaN,NaN,NaN,NaN,NaN
2,5000m,鬼頭 清仁,"18'20""04",-,NaN,NaN,18.0,20,4.0
3,5000m,大竹 康仁,"18'20""79",-,NaN,NaN,18.0,20,79.0
4,NaN,辻 航平,"16'32""20",-,NaN,NaN,16.0,32,20.0
5,NaN,小林 雅彦,DNS,-,NaN,NaN,NaN,NaN,NaN
6,NaN,松井 将器,"14'21""57",PB/工大新記録,NaN,NaN,14.0,21,57.0
7,100m予選,荒川 雄斗,"11""03",+1.7,-,NaN,NaN,11,3.0
8,NaN,米川 晴義,"11""10",+2.3,-,NaN,NaN,11,10.0
9,NaN,関 駿太,"10""99",+2.8,-,NaN,NaN,10,99.0


In [21]:
track3 = copy.deepcopy(track2)

for i in range(len(track3)):
    discipline = track3.iloc[i]['種目']
    if isinstance(discipline, str):
        temp_dis = discipline
    
    else:
        track3['種目'][i] = temp_dis

track3['区分'] = [np.nan for i in range(len(track3))]

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
track4 = copy.deepcopy(track3)

for i in range(len(track4)):
    discipline = track4.iloc[i]['種目']
    
    if re.search(r'H', discipline):
        if len(discipline.split('H')) > 1 and discipline.split('H')[1] != '':
                track4['区分'][i] = discipline.split('H')[1].replace(' ', '')
                track4['種目'][i] = discipline.replace(discipline.split('H')[1], '')
    
    elif re.search(r'C', discipline):
        if len(discipline.split('C')) > 1 and discipline.split('C')[1] != '':
                track4['区分'][i] = discipline.split('C')[1].replace(' ', '')
                track4['種目'][i] = discipline.replace(discipline.split('C')[1], '')

    elif re.search(r'TR', discipline):
        if len(discipline.split('R')) > 1 and discipline.split('m')[1] != '':
                track4['区分'][i] = discipline.split('m')[1].replace(' ', '')
                track4['種目'][i] = discipline.replace(discipline.split('m')[1], '')

    elif re.search(r'mR', discipline):
        if len(discipline.split('R')) > 1 and discipline.split('R')[1] != '':
                track4['区分'][i] = discipline.split('R')[1].replace(' ', '')
                track4['種目'][i] = discipline.replace(discipline.split('R')[1], '')
                
    elif len(discipline.split('m')) > 1 and discipline.split('m')[1] != '':
        track4['区分'][i] = discipline.split('m')[1].replace(' ', '')
        track4['種目'][i] = discipline.replace(discipline.split('m')[1], '')

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/

In [23]:
track4

,種目,氏名,記録,風,備考,時間,分,秒,秒以下,区分
0,1500m,松井 将器,"3'57""28",PB/歴代4位,NaN,NaN,3.0,57,28.0,NaN
1,1500m,柴田 幸樹,DNS,-,NaN,NaN,NaN,NaN,NaN,NaN
2,5000m,鬼頭 清仁,"18'20""04",-,NaN,NaN,18.0,20,4.0,NaN
3,5000m,大竹 康仁,"18'20""79",-,NaN,NaN,18.0,20,79.0,NaN
4,5000m,辻 航平,"16'32""20",-,NaN,NaN,16.0,32,20.0,NaN
5,5000m,小林 雅彦,DNS,-,NaN,NaN,NaN,NaN,NaN,NaN
6,5000m,松井 将器,"14'21""57",PB/工大新記録,NaN,NaN,14.0,21,57.0,NaN
7,100m,荒川 雄斗,"11""03",+1.7,-,NaN,NaN,11,3.0,予選
8,100m,米川 晴義,"11""10",+2.3,-,NaN,NaN,11,10.0,予選
9,100m,関 駿太,"10""99",+2.8,-,NaN,NaN,10,99.0,予選


In [24]:
track5 = copy.deepcopy(track4)

for i in range(len(track5)):
    wind = track5.iloc[i]['風']
    if isinstance(wind, float) and math.isnan(wind):
        continue
    else:
        if re.search(r'PB', wind) or re.search(r'位', wind) or re.search(r'Q', wind) or re.search(r'q', wind):
            track5['備考'][i] = wind
            track5['風'][i] = np.nan
            
track5

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,種目,氏名,記録,風,備考,時間,分,秒,秒以下,区分
0,1500m,松井 将器,"3'57""28",NaN,PB/歴代4位,NaN,3.0,57,28.0,NaN
1,1500m,柴田 幸樹,DNS,-,NaN,NaN,NaN,NaN,NaN,NaN
2,5000m,鬼頭 清仁,"18'20""04",-,NaN,NaN,18.0,20,4.0,NaN
3,5000m,大竹 康仁,"18'20""79",-,NaN,NaN,18.0,20,79.0,NaN
4,5000m,辻 航平,"16'32""20",-,NaN,NaN,16.0,32,20.0,NaN
5,5000m,小林 雅彦,DNS,-,NaN,NaN,NaN,NaN,NaN,NaN
6,5000m,松井 将器,"14'21""57",NaN,PB/工大新記録,NaN,14.0,21,57.0,NaN
7,100m,荒川 雄斗,"11""03",+1.7,-,NaN,NaN,11,3.0,予選
8,100m,米川 晴義,"11""10",+2.3,-,NaN,NaN,11,10.0,予選
9,100m,関 駿太,"10""99",+2.8,-,NaN,NaN,10,99.0,予選


In [25]:
track6 = copy.deepcopy(track5)

for i in range(len(track6)):
    record = track6.iloc[i]['記録']
    name = track6.iloc[i]['氏名']
    if record == 'DNS' or record == 'DNF' or record == 'DQ' or record == 'DSQ':
        track6['備考'][i] = record
    if re.search('　', name) or re.search(' ', name):
        track6['氏名'][i] = name.replace('　', '').replace(' ', '')
        
track6

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,種目,氏名,記録,風,備考,時間,分,秒,秒以下,区分
0,1500m,松井将器,"3'57""28",NaN,PB/歴代4位,NaN,3.0,57,28.0,NaN
1,1500m,柴田幸樹,DNS,-,DNS,NaN,NaN,NaN,NaN,NaN
2,5000m,鬼頭清仁,"18'20""04",-,NaN,NaN,18.0,20,4.0,NaN
3,5000m,大竹康仁,"18'20""79",-,NaN,NaN,18.0,20,79.0,NaN
4,5000m,辻航平,"16'32""20",-,NaN,NaN,16.0,32,20.0,NaN
5,5000m,小林雅彦,DNS,-,DNS,NaN,NaN,NaN,NaN,NaN
6,5000m,松井将器,"14'21""57",NaN,PB/工大新記録,NaN,14.0,21,57.0,NaN
7,100m,荒川雄斗,"11""03",+1.7,-,NaN,NaN,11,3.0,予選
8,100m,米川晴義,"11""10",+2.3,-,NaN,NaN,11,10.0,予選
9,100m,関駿太,"10""99",+2.8,-,NaN,NaN,10,99.0,予選


In [30]:
track6.to_csv(p_name + '_track.csv', encoding='shift_jis', sep=',')

In [44]:
field2 = copy.deepcopy(field)

for i in range(len(field2)):
    line = field.iloc[i]['記録']

    if re.search('m', line):
        field2['記録'][i] = line.replace('m', '.')

In [45]:
field3 = copy.deepcopy(field2)

for i in range(len(field3)):
    discipline = field3.iloc[i]['種目']
    if isinstance(discipline, str):
        temp_dis = discipline
    
    else:
        field3['種目'][i] = temp_dis

field3['区分'] = [np.nan for i in range(len(field3))]

In [46]:
field4 = copy.deepcopy(field3)

for i in range(len(field4)):
    discipline = field4.iloc[i]['種目']
    
    if re.search(r'跳', discipline):
        if len(discipline.split('跳')) > 1 and discipline.split('跳')[1] != '':
                field4['区分'][i] = discipline.split('跳')[1].replace(' ', '')
                field4['種目'][i] = discipline.replace(discipline.split('跳')[1], '')
    
    elif re.search(r'投', discipline):
        if len(discipline.split('投')) > 1 and discipline.split('投')[1] != '':
                field4['区分'][i] = discipline.split('投')[1].replace(' ', '')
                field4['種目'][i] = discipline.replace(discipline.split('投')[1], '')

/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amano/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
field5 = copy.deepcopy(field4)

for i in range(len(field5)):
    wind = field5.iloc[i]['風']
    if isinstance(wind, float) and math.isnan(wind):
        continue
    else:
        if re.search(r'PB', wind) or re.search(r'位', wind) or re.search(r'Q', wind) or re.search(r'q', wind):
            field5['備考'][i] = wind
            field5['風'][i] = np.nan

In [48]:
field6 = copy.deepcopy(field5)

for i in range(len(field6)):
    record = field6.iloc[i]['記録']
    name = field6.iloc[i]['氏名']
    if record == 'DNS' or record == 'DNF' or record == 'DQ' or record == 'DSQ' or record == 'NM':
        field6['備考'][i] = record
    if re.search('　', name) or re.search(' ', name):
        field6['氏名'][i] = name.replace('　', '').replace(' ', '')
        
field6

,種目,氏名,記録,風,備考,区分
0,走高跳,足達哲也,-,-,DNS,決勝
1,走高跳,小林拓巳,NM,-,NM,NaN
2,走高跳,足達哲也,DNS,-,DNS,NaN
3,走幅跳,西嶋健人,5.64,+0.7,PB,NaN
4,走幅跳,安智潤,DNS,-,DNS,NaN
5,走高跳,小河雅広,1.90,-,-,NaN
6,走高跳,小林拓己,1.95,-,-,NaN
7,走高跳,足達哲也,NM,-,NM,NaN
8,走幅跳,西嶋健人,DNS,-,DNS,NaN
9,三段跳,小林拓己,DNS,-,DNS,NaN


In [49]:
field6.to_csv(p_name + '_field.csv', encoding='shift_jis')